<a href="https://colab.research.google.com/github/anubhavgupta1/Deep-Learning-Using-TensorFlow/blob/master/5_Training_Deep_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preprocessing**

In [1]:
!ls
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/")
!ls

sample_data
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0

In [2]:
!pip install tensorflow==1.10.0

     |████████████████████████████████| 58.4MB 76kB/s 
     |████████████████████████████████| 12.2MB 5.8MB/s 
     |████████████████████████████████| 573kB 44.8MB/s 
     |████████████████████████████████| 3.3MB 37.2MB/s 
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tifffile 2020.9.3 has requirement numpy>=1.15.1, but you'll have numpy 1.14.5 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: seaborn 0.11.0 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: pandas 1.1.4 has requirement nump

In [3]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from functools import partial

from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from matplotlib.colors import ListedColormap


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [5]:
mnist = input_data.read_data_sets("data/")
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

In [7]:
learning_rate = 0.001

In [8]:
n_epochs = 40
batch_size = 50

#**Fast Optimizers**



##**Normal GD**

In [9]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
he_init = tf.contrib.layers.variance_scaling_initializer()

In [10]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [11]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [12]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [13]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [14]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [15]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps_normal_gd/my_model_final.ckpt")

0 Test accuracy: 0.8586
1 Test accuracy: 0.8926
2 Test accuracy: 0.9079
3 Test accuracy: 0.9171
4 Test accuracy: 0.921
5 Test accuracy: 0.925
6 Test accuracy: 0.9274
7 Test accuracy: 0.9313
8 Test accuracy: 0.9327
9 Test accuracy: 0.9335
10 Test accuracy: 0.9381
11 Test accuracy: 0.9392
12 Test accuracy: 0.941
13 Test accuracy: 0.9433
14 Test accuracy: 0.9432
15 Test accuracy: 0.9453
16 Test accuracy: 0.9452
17 Test accuracy: 0.9459
18 Test accuracy: 0.9484
19 Test accuracy: 0.949
20 Test accuracy: 0.9482
21 Test accuracy: 0.9495
22 Test accuracy: 0.9508
23 Test accuracy: 0.9529
24 Test accuracy: 0.9525
25 Test accuracy: 0.954
26 Test accuracy: 0.9542
27 Test accuracy: 0.9543
28 Test accuracy: 0.9549
29 Test accuracy: 0.9561
30 Test accuracy: 0.9555
31 Test accuracy: 0.9559
32 Test accuracy: 0.9563
33 Test accuracy: 0.9577
34 Test accuracy: 0.9565
35 Test accuracy: 0.9581
36 Test accuracy: 0.9596
37 Test accuracy: 0.959
38 Test accuracy: 0.959
39 Test accuracy: 0.9607


##**Momentum GD**

In [16]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
he_init = tf.contrib.layers.variance_scaling_initializer()

In [17]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [18]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [19]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [20]:
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
    training_op = optimizer.minimize(loss)

In [21]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [22]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps_momentum/my_model_final.ckpt")

0 Test accuracy: 0.9337
1 Test accuracy: 0.9461
2 Test accuracy: 0.9526
3 Test accuracy: 0.9549
4 Test accuracy: 0.9593
5 Test accuracy: 0.9622
6 Test accuracy: 0.9633
7 Test accuracy: 0.9632
8 Test accuracy: 0.9658
9 Test accuracy: 0.9671
10 Test accuracy: 0.9681
11 Test accuracy: 0.9713
12 Test accuracy: 0.9712
13 Test accuracy: 0.9702
14 Test accuracy: 0.9724
15 Test accuracy: 0.9716
16 Test accuracy: 0.9716
17 Test accuracy: 0.9709
18 Test accuracy: 0.9715
19 Test accuracy: 0.9733
20 Test accuracy: 0.9729
21 Test accuracy: 0.9716
22 Test accuracy: 0.9716
23 Test accuracy: 0.9721
24 Test accuracy: 0.9733
25 Test accuracy: 0.9726
26 Test accuracy: 0.9723
27 Test accuracy: 0.9736
28 Test accuracy: 0.9725
29 Test accuracy: 0.9732
30 Test accuracy: 0.9739
31 Test accuracy: 0.9728
32 Test accuracy: 0.974
33 Test accuracy: 0.9746
34 Test accuracy: 0.9737
35 Test accuracy: 0.9735
36 Test accuracy: 0.9751
37 Test accuracy: 0.9739
38 Test accuracy: 0.9744
39 Test accuracy: 0.9746


##**Nestrov**

In [23]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
he_init = tf.contrib.layers.variance_scaling_initializer()

In [24]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [25]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [26]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [27]:
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)

In [28]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [29]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps_nestrov/my_model_final.ckpt")

0 Test accuracy: 0.9321
1 Test accuracy: 0.9453
2 Test accuracy: 0.9516
3 Test accuracy: 0.9586
4 Test accuracy: 0.9606
5 Test accuracy: 0.9625
6 Test accuracy: 0.9635
7 Test accuracy: 0.9648
8 Test accuracy: 0.9664
9 Test accuracy: 0.9678
10 Test accuracy: 0.9675
11 Test accuracy: 0.9651
12 Test accuracy: 0.9701
13 Test accuracy: 0.968
14 Test accuracy: 0.9679
15 Test accuracy: 0.9704
16 Test accuracy: 0.9688
17 Test accuracy: 0.9701
18 Test accuracy: 0.9722
19 Test accuracy: 0.97
20 Test accuracy: 0.973
21 Test accuracy: 0.9711
22 Test accuracy: 0.972
23 Test accuracy: 0.9708
24 Test accuracy: 0.9714
25 Test accuracy: 0.9712
26 Test accuracy: 0.9714
27 Test accuracy: 0.9698
28 Test accuracy: 0.969
29 Test accuracy: 0.9713
30 Test accuracy: 0.971
31 Test accuracy: 0.9722
32 Test accuracy: 0.9733
33 Test accuracy: 0.9733
34 Test accuracy: 0.9712
35 Test accuracy: 0.9722
36 Test accuracy: 0.9728
37 Test accuracy: 0.9725
38 Test accuracy: 0.9719
39 Test accuracy: 0.9724


##**Adagrad**

In [30]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
he_init = tf.contrib.layers.variance_scaling_initializer()

In [31]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [32]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [33]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [34]:
with tf.name_scope("train"):
    optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [35]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [36]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps_adagrad/my_model_final.ckpt")

0 Test accuracy: 0.8903
1 Test accuracy: 0.9108
2 Test accuracy: 0.9194
3 Test accuracy: 0.9258
4 Test accuracy: 0.9296
5 Test accuracy: 0.9331
6 Test accuracy: 0.9347
7 Test accuracy: 0.9377
8 Test accuracy: 0.9384
9 Test accuracy: 0.9401
10 Test accuracy: 0.9416
11 Test accuracy: 0.9419
12 Test accuracy: 0.9427
13 Test accuracy: 0.9441
14 Test accuracy: 0.9455
15 Test accuracy: 0.9452
16 Test accuracy: 0.9462
17 Test accuracy: 0.9465
18 Test accuracy: 0.9476
19 Test accuracy: 0.949
20 Test accuracy: 0.9483
21 Test accuracy: 0.9497
22 Test accuracy: 0.9508
23 Test accuracy: 0.9508
24 Test accuracy: 0.951
25 Test accuracy: 0.9525
26 Test accuracy: 0.953
27 Test accuracy: 0.9534
28 Test accuracy: 0.9538
29 Test accuracy: 0.9546
30 Test accuracy: 0.9544
31 Test accuracy: 0.9549
32 Test accuracy: 0.9555
33 Test accuracy: 0.9559
34 Test accuracy: 0.9567
35 Test accuracy: 0.9565
36 Test accuracy: 0.9565
37 Test accuracy: 0.9572
38 Test accuracy: 0.9573
39 Test accuracy: 0.957


##**RMS Prop**

In [37]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
he_init = tf.contrib.layers.variance_scaling_initializer()

In [38]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [39]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [40]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [41]:
with tf.name_scope("train"):
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate,momentum=0.9, decay=0.9, epsilon=1e-10)
    training_op = optimizer.minimize(loss)

In [42]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [43]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps_rms_prop/my_model_final.ckpt")

0 Test accuracy: 0.9275
1 Test accuracy: 0.9454
2 Test accuracy: 0.9457
3 Test accuracy: 0.9532
4 Test accuracy: 0.9548
5 Test accuracy: 0.9393
6 Test accuracy: 0.9602
7 Test accuracy: 0.9499
8 Test accuracy: 0.9634
9 Test accuracy: 0.9474
10 Test accuracy: 0.9601
11 Test accuracy: 0.9469
12 Test accuracy: 0.952
13 Test accuracy: 0.9569
14 Test accuracy: 0.9456
15 Test accuracy: 0.9516
16 Test accuracy: 0.9484
17 Test accuracy: 0.9487
18 Test accuracy: 0.9402
19 Test accuracy: 0.9304
20 Test accuracy: 0.9503
21 Test accuracy: 0.9482
22 Test accuracy: 0.9326
23 Test accuracy: 0.8531
24 Test accuracy: 0.9257
25 Test accuracy: 0.8474
26 Test accuracy: 0.8381
27 Test accuracy: 0.7875
28 Test accuracy: 0.8545
29 Test accuracy: 0.8462
30 Test accuracy: 0.8529
31 Test accuracy: 0.8016
32 Test accuracy: 0.8231
33 Test accuracy: 0.8422
34 Test accuracy: 0.7749
35 Test accuracy: 0.5961
36 Test accuracy: 0.7286
37 Test accuracy: 0.8469
38 Test accuracy: 0.8521
39 Test accuracy: 0.8532


##**Adam**

In [44]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
he_init = tf.contrib.layers.variance_scaling_initializer()

In [45]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [46]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [47]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [48]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [49]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [50]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps_adam/my_model_final.ckpt")

0 Test accuracy: 0.9478
1 Test accuracy: 0.9526
2 Test accuracy: 0.9571
3 Test accuracy: 0.9635
4 Test accuracy: 0.9659
5 Test accuracy: 0.9653
6 Test accuracy: 0.9659
7 Test accuracy: 0.9681
8 Test accuracy: 0.9686
9 Test accuracy: 0.9685
10 Test accuracy: 0.9734
11 Test accuracy: 0.9729
12 Test accuracy: 0.9674
13 Test accuracy: 0.9762
14 Test accuracy: 0.9743
15 Test accuracy: 0.9738
16 Test accuracy: 0.9715
17 Test accuracy: 0.976
18 Test accuracy: 0.975
19 Test accuracy: 0.9732
20 Test accuracy: 0.9729
21 Test accuracy: 0.9694
22 Test accuracy: 0.975
23 Test accuracy: 0.9748
24 Test accuracy: 0.9748
25 Test accuracy: 0.9763
26 Test accuracy: 0.9732
27 Test accuracy: 0.9746
28 Test accuracy: 0.9735
29 Test accuracy: 0.9751
30 Test accuracy: 0.9711
31 Test accuracy: 0.9754
32 Test accuracy: 0.9725
33 Test accuracy: 0.978
34 Test accuracy: 0.9759
35 Test accuracy: 0.9781
36 Test accuracy: 0.9783
37 Test accuracy: 0.9771
38 Test accuracy: 0.9763
39 Test accuracy: 0.979
